In [1]:
import cv2
import os
import sys
import numpy as np
import argparse
sys.path.append(os.getcwd())
import utils.video as uv
from os import path, listdir
from os.path import isfile, join, exists

In [9]:
def chequear_carac_video(video_path):
    """Muestra las características principales de un video"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("No se pudo abrir el video")
        return None
    
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  
    fps    = cap.get(cv2.CAP_PROP_FPS)               
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  
    duration = frames / fps                           

    print(f"Video: {os.path.basename(video_path)}")
    print(f"Resolución: {width}x{height}")
    print(f"FPS: {fps}")
    print(f"Total de frames: {frames}")
    print(f"Duración: {duration/60:.2f} minutos")
    cap.release()
    return fps, width, height

In [10]:
video_path = r"./Data/AEC/Videos/RawVideo/ira_alegria.mp4"
chequear_carac_video(video_path)

Video: ira_alegria.mp4
Resolución: 1280x720
FPS: 29.97002997002997
Total de frames: 51582
Duración: 28.69 minutos


(29.97002997002997, 1280, 720)

In [64]:
video_path = r"./Historias vinetas (2)_crop.mp4"
chequear_carac_video(video_path)

Video: Historias vinetas (2)_crop.mp4
Resolución: 220x220
FPS: 29.941
Total de frames: 3045
Duración: 1.70 minutos


(29.941, 220, 220)

In [56]:
import cv2
import mediapipe as mp

def saber_coordenadas_videos(video_path, frames_to_check=30, margin=230):
    """Detecta un ROI automáticamente en los primeros frames del video usando MediaPipe"""
    cap = cv2.VideoCapture(video_path)
    mp_holistic = mp.solutions.holistic

    x_min, y_min, x_max, y_max = 9999, 9999, 0, 0
    w, h = 0, 0

    with mp_holistic.Holistic(min_detection_confidence=0.5) as holistic:
        for _ in range(frames_to_check):
            ret, frame = cap.read()
            if not ret:
                break
            h, w, _ = frame.shape
            results = holistic.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            if results.pose_landmarks:
                xs = [lm.x * w for lm in results.pose_landmarks.landmark]
                ys = [lm.y * h for lm in results.pose_landmarks.landmark]
                x_min, x_max = min(x_min, min(xs)), max(x_max, max(xs))
                y_min, y_max = min(y_min, min(ys)), max(y_max, max(ys))

    cap.release()

    # Expandir ROI con margen extra
    x_min = max(0, int(x_min - margin))
    y_min = max(0, int(y_min - margin))
    x_max = min(w, int(x_max + margin))
    y_max = min(h, int(y_max + margin))

    print(f"ROI auto detectado (con margen {margin}px): x={x_min}, y={y_min}, w={x_max-x_min}, h={y_max-y_min}")
    return x_min, x_max, y_min, y_max



In [62]:
def crop_video(input_path, output_path, target_size=(220, 220)):
    """Recorta un video en base a un ROI detectado automáticamente, redimensiona a 220x220 y guarda la salida"""
    # Verificar características
    fps, width, height = chequear_carac_video(input_path)

    # Detectar coordenadas ROI (no altero margin)
    x_min, x_max, y_min, y_max = saber_coordenadas_videos(input_path)

    # Abrir video original
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"No se pudo abrir el video: {input_path}")
        sys.exit(1)

    # Definir VideoWriter con tamaño fijo target_size
    fcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fcc, fps, target_size)

    # Procesar frame por frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        crop_frame = frame[y_min:y_max, x_min:x_max]  # recorte según ROI
        resized_frame = cv2.resize(crop_frame, target_size, interpolation=cv2.INTER_AREA)  # redimensionar

        out.write(resized_frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print(f"Video recortado y redimensionado guardado en {output_path}")


In [60]:
def final_crop_video():

    rawVideoPath = "./Data/PUCP_PSL_DGI156/Videos/original/"
    inputName = "Historias vinetas (2)"
    outputVideoPath = "./"

    # Crear carpeta de salida si no existe
    os.makedirs(outputVideoPath, exist_ok=True)

    input_path = os.path.join(rawVideoPath, inputName + ".mp4")
    output_path = os.path.join(outputVideoPath, inputName + "_crop.mp4")

    crop_video(input_path, output_path)

In [63]:
final_crop_video()

Video: Historias vinetas (2).mp4
Resolución: 1920x1080
FPS: 29.94124828948123
Total de frames: 3045
Duración: 1.69 minutos
ROI auto detectado (con margen 230px): x=451, y=0, w=1191, h=1080
Video recortado y redimensionado guardado en ./Historias vinetas (2)_crop.mp4
